In [ ]:
import os, pandas, moment, xmlmanip
os.environ['INTERFACE_CONF_FILE'] = '/home/jjorissen/interface_secrets.conf'
site_id_path = '/home/jjorissen/Documents/TangierDocs/LocationSiteIds.xlsx' 
from tangier_api import api
from tangier_api.api import ScheduleConnection, ProviderConnection, ProviderReport

In [ ]:
connection = ScheduleConnection()
sites_df = pandas.read_excel(site_id_path).dropna()

In [ ]:
today = moment.utcnow().strftime("%Y-%m-%d")
three_months_ago = moment.utcnow().add(months=-2).strftime("%Y-%m-%d")
shift_df = pandas.DataFrame()
for index, row in sites_df.iterrows():
    schedule = connection.get_schedule(site_id=row['Site ID'], start_date=three_months_ago, end_date=today)
    schema = xmlmanip.XMLSchema(schedule.encode('utf-8'))
    temp_df = pandas.DataFrame(schema.search(siteid__ne=""))
    if temp_df.shape[0] != 0:
        shift_df = shift_df.append(temp_df)

In [ ]:
report = ProviderReport(shift_df)
report.add_to_report("last_name", "first_name", key_column="empid")

In [ ]:
report.df.groupby(["empid", "providername", "provider_last_name", "provider_first_name"]).count().to_excel("who_has_npis.xlsx")

In [ ]:
report.df.columns

In [ ]:
report.df["location"]